In [1]:
pip install transformers torch pandas scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 962.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 18.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 18.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 17.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

# Load your uploaded CSV file
data = pd.read_csv("data.csv")

# Inspect the first few rows
print("Columns in CSV:", data.columns)
print(data.head())
# Look at the first few rows of the 'attendance' column



Columns in CSV: Index(['ID', 'Name', 'check-In', 'check-Out', 'courses_completed',
       'course_list', 'review_text', 'manager_rating', 'learning_summery'],
      dtype='object')
    ID               Name            check-In           check-Out  \
0  1.0       Shams Tabrez   10/8/2024 9:30:00  10/8/2024 18:00:00   
1  NaN                NaN   11/8/2024 9:00:00  11/8/2024 18:00:00   
2  NaN                NaN  12/8/2024 10:00:00  12/8/2024 18:30:00   
3  NaN                NaN  13/8/2024 10:30:00  13/8/2024 17:00:00   
4  2.0  MD Ghazanfar Alam   10/8/2024 9:00:00  10/8/2024 17:30:00   

   courses_completed                                        course_list  \
0                3.0  1. GCP Data Engineering  \n2. GEN AI \n3. GCP ...   
1                NaN                                                NaN   
2                NaN                                                NaN   
3                NaN                                                NaN   
4                1.0         

In [6]:
# It seems that there are missing names in some rows, but based on proximity, those entries could belong to Shams Tabrez.
# I'll fill in the missing names in adjacent rows with NaN values that likely belong to Shams Tabrez.

# Fill missing names based on the first non-null 'Name' value downward
data['Name'].fillna(method='ffill', inplace=True)

# Now filter the data again for Shams Tabrez and count his attendance
data_updated = data[data['Name'] == 'Shams Tabrez']
attendance_count_updated = data_updated.shape[0]
attendance_count_updated

0     10/8/2024 9:30:00
1     11/8/2024 9:00:00
2    12/8/2024 10:00:00
3    13/8/2024 10:30:00
Name: check-In, dtype: object


/tmp/ipykernel_4434/2736433662.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['Name'].fillna(method='ffill', inplace=True)


4

In [10]:
import pandas as pd

# Convert 'check-In' and 'check-Out' to datetime with the correct format
data['check-In'] = pd.to_datetime(data['check-In'], dayfirst=True, errors='coerce')
data['check-Out'] = pd.to_datetime(data['check-Out'], dayfirst=True, errors='coerce')

# Check for any failed conversions (rows with NaT)
if data['check-In'].isna().any() or data['check-Out'].isna().any():
    print("Some date-time conversions failed. Check the input format.")

# Fill missing 'Name' values using forward fill
data['Name'] = data['Name'].ffill()

# Filter the data for 'Shams Tabrez' (creating a new DataFrame to avoid 'view' issues)
shams_data = data[data['Name'] == 'Shams Tabrez'].copy()

# Calculate the duration for each day (in hours)
shams_data['Duration'] = (shams_data['check-Out'] - shams_data['check-In']).dt.total_seconds() / 3600

# Check if Shams stayed for at least 9 hours each day
shams_data['Stayed_9Hrs'] = shams_data['Duration'] >= 9

# Print details for each day (Duration and whether he stayed 9+ hours)
print(shams_data[['check-In', 'check-Out', 'Duration', 'Stayed_9Hrs']])

# Calculate the average daily stay time
average_stay = shams_data['Duration'].mean()
print(f"Average stay time: {average_stay:.2f} hours")

# Optionally: Check how many days he stayed 9+ hours
days_9hrs = shams_data['Stayed_9Hrs'].sum()
print(f"Number of days with 9+ hours: {days_9hrs}")


             check-In           check-Out  Duration  Stayed_9Hrs
0 2024-08-10 09:30:00 2024-08-10 18:00:00       8.5        False
1 2024-08-11 09:00:00 2024-08-11 18:00:00       9.0         True
2 2024-08-12 10:00:00 2024-08-12 18:30:00       8.5        False
3 2024-08-13 10:30:00 2024-08-13 17:00:00       6.5        False
Average stay time: 8.12 hours
Number of days with 9+ hours: 1
